In [1]:
import sqlite3
import pandas as pd
import os
import numpy as np  # Import numpy for numeric operations and NaN handling

# Paths and connection strings
sqlite_db_path = r"C:\Users\jcerv\Jose\podcasts\podcasts (read-only).db"

# Output file path (CSV)
output_file = 'podcasts_sample.csv'

# Connect to the SQLite database
try:
    sqlite_conn = sqlite3.connect(sqlite_db_path)
    print("Connected to SQLite database.")
except sqlite3.Error as e:
    print(f"An error occurred while connecting to SQLite: {e}")
    exit(1)
    
# Define the common filters
common_filters = """
    p.country = 'United States'
    AND p.email IS NOT NULL 
    AND p.email != ''
    AND p.language = 'English'
    AND p.rss IS NOT NULL
    AND p.has_guest_interviews = 1
    AND p.episode_count > 7
    AND strftime('%Y', p.latest_episode_pub_date) IN ('2024', '2025')
    AND (
        (p.itunes_id IS NOT NULL AND p.itunes_id != '')
        OR (p.spotify IS NOT NULL AND p.spotify != '')
    )
"""

# Query to get podcast data without genres
podcast_query = f"""
SELECT
    p.podcast_id,
    p.title,
    p.artwork_image,
    p.artwork_thumbnail,
    p.listen_score,
    p.listen_score_global_rank,
    p.rss,
    p.publisher,
    p.description,
    p.language,
    p.country,
    p.email,
    p.website,
    p.episode_count,
    p.audio_length_seconds,
    p.update_frequency_hours,
    p.latest_episode_pub_date,
    p.earliest_episode_pub_date,
    p.has_guest_interviews,
    p.has_sponsors,
    p.itunes_id,
    p.twitter,
    p.instagram,
    p.patreon,
    p.wechat,
    p.facebook,
    p.amazon_music,
    p.spotify,
    p.youtube,
    p.linkedin,
    p.wikipedia,
    p.url1,
    p.url2,
    p.url3,
    p.explicit,
    p.is_complete,
    p.type,
    p.imported_time
FROM podcasts p
WHERE {common_filters}
"""

# Query to get podcast_genres mapping
podcast_genres_query = """
SELECT
    pg.podcast_id,
    pg.genre_id
FROM podcast_genres pg
"""

# Query to get genre hierarchy
genres_query = """
SELECT
    g.id,
    g.name,
    g.parent_id
FROM genres g
"""

# Read podcast data into DataFrame
podcasts_df = pd.read_sql_query(podcast_query, sqlite_conn)

# Read podcast_genres mapping into DataFrame
podcast_genres_df = pd.read_sql_query(podcast_genres_query, sqlite_conn)

# Read genres into DataFrame
genres_df = pd.read_sql_query(genres_query, sqlite_conn)

# Close the connection
sqlite_conn.close()

# Print number of podcasts retrieved
total_podcasts = len(podcasts_df)
print(f"Total podcasts in the sample before filtering: {total_podcasts}")

# Build a genre hierarchy to map each genre to its primary genre
# First, set the index of genres_df to id for easy lookup
genres_df.set_index('id', inplace=True)

# Ensure that 'parent_id' is numeric and handle missing values
genres_df['parent_id'] = genres_df['parent_id'].astype(pd.Int64Dtype())

# Function to find the primary genre for a given genre_id
def find_primary_genre(genre_id):
    current_id = genre_id
    while current_id is not None and not pd.isna(current_id):
        try:
            parent_id = genres_df.at[current_id, 'parent_id']
        except KeyError:
            # If the genre_id is not found in genres_df, return None
            return None

        if pd.isna(parent_id) or parent_id is None:
            # Reached the top without finding parent_id = 67
            return None
        elif parent_id == 67:
            # If parent_id is 67, current genre is primary
            return genres_df.at[current_id, 'name']
        else:
            # Move up the hierarchy
            current_id = parent_id  # parent_id should be an integer or NA
    return None

# Add the genre names to podcast_genres_df
podcast_genres_df['genre_name'] = podcast_genres_df['genre_id'].map(genres_df['name'])

# Find the primary genre for each genre_id
podcast_genres_df['primary_genre'] = podcast_genres_df['genre_id'].apply(find_primary_genre)

# Remove rows where 'primary_genre' is None (could not find a primary genre)
podcast_genres_df = podcast_genres_df[podcast_genres_df['primary_genre'].notna()]

# Now, for each podcast, exclude primary_genres from secondary_genres
def get_genres(group):
    primary_genres_set = set(group['primary_genre'].dropna())
    genre_names_set = set(group['genre_name'].dropna())
    # Exclude primary genres from secondary genres
    secondary_genres_set = genre_names_set - primary_genres_set
    primary_genres_str = ', '.join(sorted(primary_genres_set))
    secondary_genres_str = ', '.join(sorted(secondary_genres_set))
    return pd.Series({'primary_genre': primary_genres_str, 'secondary_genres': secondary_genres_str})

# Apply the function to each group to combine genres for each podcast.
genres = podcast_genres_df.groupby('podcast_id').apply(get_genres).reset_index()

# Merge genres back to podcasts_df
podcasts_df = podcasts_df.merge(genres, on='podcast_id', how='inner')

# Function to choose a single podcast per email
def choose_podcast(group):
    # If there's only one podcast for this email, just return it.
    if len(group) == 1:
        return group

    # Separate out rows with a valid (non-null) listen_score.
    valid_scores = group[group['listen_score'].notna()]

    # If no rows have a valid listen_score, randomly select one from the entire group.
    if valid_scores.empty:
        return group.sample(n=1)
    
    # Otherwise, sort only the rows with valid listen_score.
    sorted_group = valid_scores.sort_values(by='listen_score')

    # Find the median position (if even, picks the lower median).
    median_index = len(sorted_group) // 2
    median_score = sorted_group.iloc[median_index]['listen_score']

    # Use np.isclose for a tolerant comparison of the listen_score.
    median_group = sorted_group[np.isclose(sorted_group['listen_score'], median_score)]
    
    # Fallback: if median_group is empty (which is unlikely), fallback to the row at median_index.
    if median_group.empty:
        median_group = sorted_group.iloc[[median_index]]
    
    # If the selection is ambiguous (i.e., more than one podcast has the median score or the original group has exactly 2),
    # randomly select one of the candidates.
    if len(median_group) > 1 or len(group) == 2:
        return median_group.sample(n=1)
    else:
        return median_group

# Group by email and apply the selection function to remove duplicate podcasts.
podcasts_df = podcasts_df.groupby('email', group_keys=False).apply(choose_podcast)

# Print number of podcasts after filtering duplicates by email.
final_podcast_count = len(podcasts_df)
print(f"Total podcasts in the sample after filtering duplicates by email: {final_podcast_count}")

# Save to CSV
podcasts_df.to_csv(output_file, index=False, encoding='utf-8')
print(f"Sample saved to {output_file}")


Connected to SQLite database.
Total podcasts in the sample before filtering: 122583


C:\Users\jcerv\AppData\Local\Temp\ipykernel_27928\4069753874.py:162: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  genres = podcast_genres_df.groupby('podcast_id').apply(get_genres).reset_index()
C:\Users\jcerv\AppData\Local\Temp\ipykernel_27928\4069753874.py:202: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  podcasts_df = podcasts_df.groupby('email', group_keys=False).apply(choose_podcast)


Total podcasts in the sample after filtering duplicates by email: 106517
Sample saved to podcasts_sample.csv


In [2]:
import pandas as pd
import json

# Read the CSV file
podcasts_df = pd.read_csv('podcasts_sample.csv')

# Read the JSONL file with UTF-8 encoding and extract podcast_ids
prod_db_ids = set()
with open('prod_db.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        prod_db_ids.add(data['podcast_id'])

# Filter the CSV to include only podcasts in prod_db.jsonl
filtered_df = podcasts_df[podcasts_df['podcast_id'].isin(prod_db_ids)]

# Count genres of interest
genre_count = 0
for genres in filtered_df['primary_genre']:
    genre_list = [g.strip() for g in str(genres).split(',')]
    if any(genre in ['Society & Culture', 'Business', 'News'] for genre in genre_list):
        genre_count += 1

print(f"Number of podcasts in filtered sample: {len(filtered_df)}")
print(f"Number of podcasts with Society & Culture, Business, or News genres: {genre_count}")


C:\Users\jcerv\AppData\Local\Temp\ipykernel_82340\4067597913.py:5: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  podcasts_df = pd.read_csv('podcasts_sample.csv')


Number of podcasts in filtered sample: 66042
Number of podcasts with Society & Culture, Business, or News genres: 32256


In [3]:
filtered_df.to_csv('podcasts_final_sample.csv', index=False)